# Setup

In [1]:
from utils import esg
from utils.ownership import parse_ownership_data
from utils.regressors import parse_regressors

In [2]:
import seaborn as sns

sns.set(style="white",
        palette="pastel",
        color_codes=True,
        rc={'figure.figsize': (12, 9) })

In [3]:
import pandas as pd
import statsmodels.api as sm

# Importing data

In [4]:
import dotenv, os

dotenv.load_dotenv()

data_path = os.path.join("..", os.environ.get("DATA_PATH"))

esg_path = os.path.join(data_path, "ESG.xlsx")
sheet_name = os.environ.get("SHEET_NAME")

own_path = os.path.join(data_path, "ownership.csv")
reg_path = os.path.join(data_path, "controls.csv")

saliency_path = os.path.join(data_path, "saliency.csv")

## Covariates

In [5]:
raw = pd.read_csv(reg_path)
covariates = parse_regressors(raw)

## ESG data

In [6]:
raw = pd.read_excel(esg_path, sheet_name = sheet_name)
get_esg = esg.yearly_esg_maker(raw)

## Ownership data

In [7]:
ownership = parse_ownership_data(pd.read_csv(own_path))
investors = ownership.columns[-7:]

In [8]:
ticker_to_name = dict(zip(ownership.TargetFirms, ownership.Tickers))

## Saliency data

In [9]:
saliency_df = pd.read_csv(saliency_path, index_col = [0, 1])

# Regression

In [10]:
def makeregressors(
    saliency_df, ownership, covariates, index, report):
    
    S = saliency_df.loc[report, index]
    companies = ownership.columns[-7:]
    X = ownership[companies]
    X.columns = [f"{company} ownership".lower() for company in companies]
    X.index = ownership['Tickers']

    for company in S.index:
        column_name = f"{company} {report.lower()}"
        X.loc[:, f"{column_name} interaction"] = X[f"{company} ownership"] * S[company] 
    
    X = sm.add_constant(X, prepend = False, has_constant='add')

    common_indices = set(X.index).intersection(covariates.index)

    X = pd.concat([X.loc[common_indices], covariates.loc[common_indices]], axis = 1)

    return X

In [11]:
X = makeregressors(saliency_df, ownership, covariates, "s-index", "STEWARDSHIP_POLICY").dropna()

/Users/andreatitton/economics/nlp-financial-env/.venv/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [12]:
y = get_esg(2020)["ESG Combined Score"] / 100
y.index = [ticker_to_name.get(f, f) for f in  y.index]
common_indices = set(y.index).intersection(set(X.index))

y = y.loc[common_indices]
X = X.loc[common_indices]

In [13]:
model = sm.RLM(y, X, M=sm.robust.norms.HuberT())

In [14]:
model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                    Robust linear Model Regression Results                    
==============================================================================
Dep. Variable:     ESG Combined Score   No. Observations:                 2096
Model:                            RLM   Df Residuals:                     2073
Method:                          IRLS   Df Model:                           22
Norm:                          HuberT                                         
Scale Est.:                       mad                                         
Cov Type:                          H1                                         
Date:                Fri, 26 Nov 2021                                         
Time:                        10:39:55                                         
No. Iterations:                    50                                         
============================================================================================================
                                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
nn ownership                                 1.4596      1.760      0.829      0.407      -1.990       4.910
blackrock ownership                         -0.4320      0.102     -4.218      0.000      -0.633      -0.231
nbim ownership                               5.8470      0.340     17.206      0.000       5.181       6.513
generali ownership                         222.6128     83.531      2.665      0.008      58.895     386.330
jpmam ownership                              0.5367      0.590      0.910      0.363      -0.619       1.693
apg ownership                                1.0048      3.873      0.259      0.795      -6.586       8.595
dws ownership                                3.7001      1.235      2.997      0.003       1.280       6.120
blackrock stewardship_policy interaction    -0.0076      0.002     -4.221      0.000      -0.011      -0.004
dws stewardship_policy interaction           0.0205      0.007      2.997      0.003       0.007       0.034
generali stewardship_policy interaction      1.3404      0.503      2.665      0.008       0.355       2.326
jpmam stewardship_policy interaction         0.0059      0.006      0.910      0.363      -0.007       0.018
nn stewardship_policy interaction            0.0094      0.011      0.829      0.407      -0.013       0.032
apg stewardship_policy interaction           0.0095      0.037      0.259      0.795      -0.062       0.081
nbim stewardship_policy interaction          0.0384      0.002     17.207      0.000       0.034       0.043
const                                        0.3517      0.008     46.751      0.000       0.337       0.366
Leverage Ratio-2016                      -1.017e-05      0.000     -0.025      0.980      -0.001       0.001
Total Debt-2016                           3.225e-08   3.32e-08      0.971      0.332   -3.28e-08    9.73e-08
Total Assets (Assets Value)-2016         -1.844e-08    1.5e-08     -1.228      0.220   -4.79e-08     1.1e-08
ROA-2016                                     0.0003      0.000      1.358      0.175      -0.000       0.001
PPE-2016                                  2.163e-07   1.48e-07      1.464      0.143   -7.32e-08    5.06e-07
Leverage Ratio-2017                         -0.0003      0.001     -0.447      0.655      -0.002       0.001
Total Debt-2017                          -2.063e-08   4.78e-08     -0.432      0.666   -1.14e-07    7.31e-08
Total Assets (Assets Value)-2017          1.784e-08   1.74e-08      1.025      0.305   -1.63e-08     5.2e-08
ROA-2017                                     0.0007      0.000      2.314      0.021       0.000       0.001
PPE-2017                                 -2.415e-07   1.72e-07     -1.402      0.161   -5.79e-07     9.6e-08
Leverage Ratio-2018            